# Notes to Improve Workflow

In [85]:
# Setup matplotlib to plot inline (within the notebook)
%matplotlib inline
# Import the pyplot module of Matplotlib as plt
import matplotlib.pyplot as plt
# Import pandas under the abbreviation 'pd'
import pandas as pd
# Import NumPy under the abbreviation 'np'
import numpy as np

#Notes for efficienccy:
#shift + tab when cursor is in arguments to bring up its documentation
#ex: pd.DataFrame(Shift+tab here)
#recall: Json == Dictionary
#to know functions of pandas:
dir(pd)

['BooleanDtype',
 'Categorical',
 'CategoricalDtype',
 'CategoricalIndex',
 'DataFrame',
 'DateOffset',
 'DatetimeIndex',
 'DatetimeTZDtype',
 'ExcelFile',
 'ExcelWriter',
 'Flags',
 'Float32Dtype',
 'Float64Dtype',
 'Float64Index',
 'Grouper',
 'HDFStore',
 'Index',
 'IndexSlice',
 'Int16Dtype',
 'Int32Dtype',
 'Int64Dtype',
 'Int64Index',
 'Int8Dtype',
 'Interval',
 'IntervalDtype',
 'IntervalIndex',
 'MultiIndex',
 'NA',
 'NaT',
 'NamedAgg',
 'Period',
 'PeriodDtype',
 'PeriodIndex',
 'RangeIndex',
 'Series',
 'SparseDtype',
 'StringDtype',
 'Timedelta',
 'TimedeltaIndex',
 'Timestamp',
 'UInt16Dtype',
 'UInt32Dtype',
 'UInt64Dtype',
 'UInt64Index',
 'UInt8Dtype',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__docformat__',
 '__file__',
 '__getattr__',
 '__git_version__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 '_config',
 '_hashtable',
 '_is_numpy_dev',
 '_lib',
 '_libs',
 '_np_version_under1p18',
 '_testing',
 '_tslib',
 '_typing',
 

# Reading The Data

In [86]:
# using pandas and read_csv()
acquis = pd.read_csv("datasets/startups/acquisitions.csv")
additions = pd.read_csv("datasets/startups/additions.csv")
companies = pd.read_csv("datasets/startups/companies.csv")
invests = pd.read_csv("datasets/startups/investments.csv")
rounds = pd.read_csv("datasets/startups/rounds.csv")
dfAll = [acquis, additions, companies, invests, rounds]

In [87]:
#shapes (rows x cols) of each dataset
for ele in dfAll:
    print(ele.shape)

(18969, 18)
(2213, 4)
(66368, 14)
(168647, 18)
(114949, 12)


# Samples of the datasets

In [88]:
acquis.sample(5).transpose()

,3689,1648,18623,9821,1485
company_permalink,/organization/codo,/organization/audiomicro,/organization/wms,/organization/lumena-pharmaceuticals,/organization/asera
company_name,Cobook,AudioMicro,WMS,Lumena Pharmaceuticals,Asera
company_category_list,Contact Management|Software,Artists Globally|Audio|Games|Music,Design|Entertainment|Games|Online Gaming|Services,Biotechnology,B2B|Business Services|Internet
company_country_code,LVA,USA,USA,USA,USA
company_state_code,25,CA,IL,CA,CA
company_region,Riga,Los Angeles,Chicago,San Diego,SF Bay Area
company_city,Riga,Sherman Oaks,Waukegan,San Diego,Belmont
acquirer_permalink,/organization/fullcontact,/organization/zealot-network,/organization/scientific-games-corporation,/organization/shire,/organization/seec
acquirer_name,FullContact,Zealot Networks,Scientific Games Corporation,Shire,SEEC
acquirer_category_list,Contact Management|Databases|Developer APIs|En...,Business Services|Digital Media|Media,Gamification|Lotteries|Loyalty Programs|Mobile,Biotechnology|Medical Devices|Pharmaceuticals,Software


In [89]:
additions.sample(5).transpose()

,174,1444,1506,1175,1441
content,Acquisitions,Rounds: debt_financing,Rounds: debt_financing,Rounds,Rounds: debt_financing
month_str,2005-09,2010-03,2015-05,2009-04,2009-12
year_str,2005,2010,2015,2009,2009
value,37,47,122,270,67


In [90]:
companies.sample(5).transpose()

,49704,15657,32639,39478,23753
permalink,/organization/sayduck,/organization/digital-fuel,/organization/liquidcool-solutions,/organization/nlife-therapeutics,/organization/grabcad
name,Sayduck,Digital Fuel,LiquidCool Solutions,nLife Therapeutics,GrabCAD
homepage_url,http://www.sayduck.com,http://www.digitalfuel.com,http://www.liquidcoolsolutions.com,NaN,http://www.grabcad.com
category_list,3D|Augmented Reality|Mobile|Mobile Commerce,Software,Manufacturing|Mechanical Solutions,Biotechnology|Ediscovery,CAD|Curated Web|Finance|Manufacturing
funding_total_usd,1477282,13000000,5182400,9679942,13618793
status,operating,acquired,operating,operating,acquired
country_code,FIN,USA,USA,ESP,USA
state_code,13,CA,MN,51,MA
region,Helsinki,SF Bay Area,"Rochester, Minnesota",Granada,Boston
city,Helsinki,San Mateo,Rochester,Granada,Cambridge


In [91]:
invests.sample(5).transpose()

,18967,82692,27230,5442,6606
company_permalink,/organization/bitinstant,/organization/livevault,/organization/cedar-point-communications,/organization/air-asia-indonesia,/organization/aliz-pharma
company_name,BitInstant,Livevault,Cedar Point Communications,Air Asia,Alizé Pharma
company_category_list,Fashion,Computers|Services|Software,Enterprise Software,Online Travel|Service Providers|Travel & Tourism,Biotechnology
company_country_code,USA,USA,USA,MYS,FRA
company_state_code,NY,MA,TX,4,B9
company_region,New York City,Boston,Austin,MYS - Other,Lyon
company_city,New York,Marlborough,Del Valle,Melaka,Écully
investor_permalink,/organization/winklevoss-capital,/organization/atlas-venture,/organization/focus-ventures,/organization/crescent-venture-investors,/organization/bpifrance
investor_name,Winklevoss Capital,Atlas Venture,Focus Ventures,Crescent Venture Investors,Bpifrance
investor_country_code,USA,USA,USA,USA,FRA


In [92]:
rounds.sample(5).transpose()

,28349,68112,39068,114782,28040
company_permalink,/organization/doapp,/organization/ninsight-broadcast,/organization/generate,/organization/zumbox,/organization/discera
company_name,DoApp,Ninsight Broadcast,Generate,Zumbox,Discera
company_category_list,Mobile,Software,Career Management|Entertainment|Games|Product ...,Curated Web|Internet,Semiconductors
company_country_code,USA,FRA,USA,USA,USA
company_state_code,MN,A8,CA,CA,CA
company_region,Minneapolis,Paris,Los Angeles,Los Angeles,SF Bay Area
company_city,Minneapolis,Issy-les-moulineaux,Santa Monica,Los Angeles,San Jose
funding_round_permalink,/funding-round/55e505d510e4417d3baf4003044d88d3,/funding-round/2a11e046ae96bb0784df16e67426aab9,/funding-round/e85f6969fce74a58e2d9f1a2b3c52e2a,/funding-round/4e3d60192265acafca017492e9c61780,/funding-round/b290d6fc497bc9544bd6be873d3e8731
funding_round_type,angel,venture,venture,venture,venture
funding_round_code,NaN,A,A,C,C


In [93]:
#getting all (unique) columns of the datasets
uniqueCols = set() #or: instead of "{}", "set()"
for ele in dfAll:
    cols = ele.columns.tolist() #".tolist()" to make it a list (side note: ".values" to make it a numpy.ndarray)
    
    uniqueCols.update(cols) #use ".add()" for elements, ".update()" for lists 

In [94]:
#sorting the columns alphabetically
tempList = list(uniqueCols)
tempList.sort()
uniqueCols = tempList
uniqueCols

['acquired_at',
 'acquired_month',
 'acquirer_category_list',
 'acquirer_city',
 'acquirer_country_code',
 'acquirer_name',
 'acquirer_permalink',
 'acquirer_region',
 'acquirer_state_code',
 'category_list',
 'city',
 'company_category_list',
 'company_city',
 'company_country_code',
 'company_name',
 'company_permalink',
 'company_region',
 'company_state_code',
 'content',
 'country_code',
 'first_funding_at',
 'founded_at',
 'funded_at',
 'funding_round_code',
 'funding_round_permalink',
 'funding_round_type',
 'funding_rounds',
 'funding_total_usd',
 'homepage_url',
 'investor_city',
 'investor_country_code',
 'investor_name',
 'investor_permalink',
 'investor_region',
 'investor_state_code',
 'last_funding_at',
 'month_str',
 'name',
 'permalink',
 'price_amount',
 'price_currency_code',
 'raised_amount_usd',
 'region',
 'state_code',
 'status',
 'value',
 'year_str']

# Seeing the columns in the datasets

In [95]:
def datasetsCols(dfList, datasetNames, pad=""):
    """
    Takes dataframes and returns a dataframe (df) with each column having a df's
    column names. The "pad" is to make sure all lists of columns have 
    the same length padded out with "pad"
    Example
    pad = "XX", datasetNames = ["cars", "planes"]
    dfList = [carsDF, planesDF]
    returned data frame:
            cars            planes
    0       numOfWheels     numOfWings
    1       manufacturer    manufacturer
    2       make            XX
    """
    cols = []
    maxArrayLen = 0
    for i, df in enumerate(dfList):
        cols.append(df.columns.tolist())
        maxArrayLen = max(maxArrayLen, len(cols[i]))
    
    dictCsvs = {}
    for i, df in enumerate(dfList):
        cols[i] += [pad] * (maxArrayLen - len(cols[i])) #padding the lists to make them have equal lengths 
        dictCsvs.update({datasetNames[i] : cols[i]})

    return pd.DataFrame(dictCsvs)

In [96]:
lstCsvNames = ["acquisitions", "additions", "companies", "investments", "rounds"]

dfAllCols = datasetsCols(dfAll, lstCsvNames)
dfAllCols

,acquisitions,additions,companies,investments,rounds
0,company_permalink,content,permalink,company_permalink,company_permalink
1,company_name,month_str,name,company_name,company_name
2,company_category_list,year_str,homepage_url,company_category_list,company_category_list
3,company_country_code,value,category_list,company_country_code,company_country_code
4,company_state_code,,funding_total_usd,company_state_code,company_state_code
5,company_region,,status,company_region,company_region
6,company_city,,country_code,company_city,company_city
7,acquirer_permalink,,state_code,investor_permalink,funding_round_permalink
8,acquirer_name,,region,investor_name,funding_round_type
9,acquirer_category_list,,city,investor_country_code,funding_round_code


# Data Cleaning

First, lets change the columns in the `companies` dataset <br>
to be the similar to all other datasets:

In [97]:
companies.rename(columns = {'permalink' : 'company_permalink', 
                            'name' : 'company_name',
                            'category_list' : 'company_category_list',
                            'country_code' : 'company_country_code',
                            'state_code' : 'company_state_code',
                            'region' : 'company_region',
                            'city' : 'company_city'}, inplace = True)

In [98]:
dfAll[2] = companies
dfAllCols = datasetsCols(dfAll, lstCsvNames)
dfAllCols

,acquisitions,additions,companies,investments,rounds
0,company_permalink,content,company_permalink,company_permalink,company_permalink
1,company_name,month_str,company_name,company_name,company_name
2,company_category_list,year_str,homepage_url,company_category_list,company_category_list
3,company_country_code,value,company_category_list,company_country_code,company_country_code
4,company_state_code,,funding_total_usd,company_state_code,company_state_code
5,company_region,,status,company_region,company_region
6,company_city,,company_country_code,company_city,company_city
7,acquirer_permalink,,company_state_code,investor_permalink,funding_round_permalink
8,acquirer_name,,company_region,investor_name,funding_round_type
9,acquirer_category_list,,company_city,investor_country_code,funding_round_code


Let's start by analyzing `additions` dataset, as it has unusual format of columns

## "additions" Dataset

In [99]:
additions.sample(5)

,content,month_str,year_str,value
131,Acquisitions,2002-02,2002,7
766,Profiles,2009-11,2009,3224
1743,Rounds: post_ipo_equity,2015-02,2015,25
1655,Rounds: non_equity_assistance,2014-12,2014,67
597,IPOs,2004-06,2004,44


In [100]:
additions["content"].unique()

array(['Acquisitions', 'IPOs', 'Profiles', 'Profiles: Organizations',
       'Profiles: People', 'Profiles: Products', 'Rounds',
       'Rounds: angel', 'Rounds: convertible_note',
       'Rounds: debt_financing', 'Rounds: equity_crowdfunding',
       'Rounds: grant', 'Rounds: non_equity_assistance',
       'Rounds: post_ipo_debt', 'Rounds: post_ipo_equity',
       'Rounds: private_equity', 'Rounds: product_crowdfunding',
       'Rounds: secondary_market', 'Rounds: seed', 'Rounds: undisclosed',
       'Rounds: venture'], dtype=object)

In [101]:
dfAllCols

,acquisitions,additions,companies,investments,rounds
0,company_permalink,content,company_permalink,company_permalink,company_permalink
1,company_name,month_str,company_name,company_name,company_name
2,company_category_list,year_str,homepage_url,company_category_list,company_category_list
3,company_country_code,value,company_category_list,company_country_code,company_country_code
4,company_state_code,,funding_total_usd,company_state_code,company_state_code
5,company_region,,status,company_region,company_region
6,company_city,,company_country_code,company_city,company_city
7,acquirer_permalink,,company_state_code,investor_permalink,funding_round_permalink
8,acquirer_name,,company_region,investor_name,funding_round_type
9,acquirer_category_list,,company_city,investor_country_code,funding_round_code


In [102]:
additions.shape

(2213, 4)

In [103]:
companies.shape, rounds.shape

((66368, 14), (114949, 12))

So apparently there are no columns that could be used to join with the other datasets. <br>
Furthermore, the "value" column is too vague to be useful. <br>
Therefore, the "additions" dataset will be discarded.

In [104]:
dfAll.pop(1)
lstCsvNames.pop(1)
len(dfAll), lstCsvNames

(4, ['acquisitions', 'companies', 'investments', 'rounds'])

## Data Cleaning (Cont.)

Now, let's see the null values for each dataset's columns:

In [105]:
def datasetsNulls(dfList, datasetNames, nullCol="nulls_", pad=""):
    """
    Use this when you want to display a column of column names,
    then a column of the null values, and repeat that for each dataset.
    Returns a dataframe
    """
    cols = []
    nulls = []
    maxArrayLen = 0
    for i, df in enumerate(dfList):
        cols.append(df.columns.tolist())
        nulls.append(df.isnull().sum().tolist())
        maxArrayLen = max(maxArrayLen, len(cols[i]))
    
    dictCsvs = {}
    for i, df in enumerate(dfList):
        cols[i] += [pad] * (maxArrayLen - len(cols[i])) #padding the lists to make them have equal lengths 
        nulls[i] += [None] *  (maxArrayLen - len(nulls[i]))
        dictCsvs[datasetNames[i]] = cols[i]
        dictCsvs[nullCol + str(i+1)] = nulls[i]
    
    return pd.DataFrame(dictCsvs)

In [106]:
dfAllNulls = datasetsNulls(dfAll, lstCsvNames)
dfAllNulls

,acquisitions,nulls_1,companies,nulls_2,investments,nulls_3,rounds,nulls_4
0,company_permalink,0,company_permalink,0.0,company_permalink,0,company_permalink,0.0
1,company_name,1,company_name,1.0,company_name,1,company_name,1.0
2,company_category_list,4984,homepage_url,5058.0,company_category_list,3632,company_category_list,3410.0
3,company_country_code,4669,company_category_list,3148.0,company_country_code,10521,company_country_code,8678.0
4,company_state_code,4949,funding_total_usd,0.0,company_state_code,13431,company_state_code,10946.0
5,company_region,4911,status,0.0,company_region,12485,company_region,10167.0
6,company_city,4911,company_country_code,6958.0,company_city,12483,company_city,10164.0
7,acquirer_permalink,1,company_state_code,8547.0,investor_permalink,12,funding_round_permalink,0.0
8,acquirer_name,1,company_region,8030.0,investor_name,12,funding_round_type,0.0
9,acquirer_category_list,2389,company_city,8028.0,investor_country_code,30939,funding_round_code,83809.0


Alternatively, we can combine all columns to be the dataframe's index:

In [107]:
#Alternatively:
def datasetsNullsSameIndex(datasetsNames, *args):
    """
    Use this when multiple datasets have a lot of columns
    with the same names.
    Returns a dataframe with the index value being the 
    columns of all datasets
    """
    dic = {}
    for i, df in enumerate(args):
        dic[datasetsNames[i]] = df.isnull().sum()
    return pd.DataFrame(dic)

In [108]:
datasetsNullsSameIndex(lstCsvNames, *dfAll)

,acquisitions,companies,investments,rounds
acquired_at,13.0,NaN,NaN,NaN
acquired_month,15.0,NaN,NaN,NaN
acquirer_category_list,2389.0,NaN,NaN,NaN
acquirer_city,1360.0,NaN,NaN,NaN
acquirer_country_code,1171.0,NaN,NaN,NaN
acquirer_name,1.0,NaN,NaN,NaN
acquirer_permalink,1.0,NaN,NaN,NaN
acquirer_region,1360.0,NaN,NaN,NaN
acquirer_state_code,1436.0,NaN,NaN,NaN
company_category_list,4984.0,3148.0,3632.0,3410.0


Notice that there is one row that doesn't contain a company name, <br>
so let's validate that it is the same company across all datasets, <br>
so we can remove it:

In [109]:
datasetsCols(dfAll, lstCsvNames)

,acquisitions,companies,investments,rounds
0,company_permalink,company_permalink,company_permalink,company_permalink
1,company_name,company_name,company_name,company_name
2,company_category_list,homepage_url,company_category_list,company_category_list
3,company_country_code,company_category_list,company_country_code,company_country_code
4,company_state_code,funding_total_usd,company_state_code,company_state_code
5,company_region,status,company_region,company_region
6,company_city,company_country_code,company_city,company_city
7,acquirer_permalink,company_state_code,investor_permalink,funding_round_permalink
8,acquirer_name,company_region,investor_name,funding_round_type
9,acquirer_category_list,company_city,investor_country_code,funding_round_code


In [110]:
#company_permalink of each row that doesn't have a company name:
for df in dfAll:
    print(df[df['company_name'].isnull()]['company_permalink'].values)

['mendatio"']
['/organization/tell-it-in']
['/organization/tell-it-in']
['/organization/tell-it-in']


So apparently, the `acquisitions` dataset has a different permalink that has no company name, <br>
So we could remove these two companies, as the total number of companies are big enough to get the insight that we want: 

Now, let's see the `dtypes` of each column:

In [111]:
def datasetsDtypes(dfList, datasetNames, dtypesCol="dtypes_", pad=""):
    cols = []
    nulls = []
    maxArrayLen = 0
    for i, df in enumerate(dfList):
        cols.append(df.columns.tolist())
        nulls.append(df.dtypes.tolist())
        maxArrayLen = max(maxArrayLen, len(cols[i]))
    
    dictCsvs = {}
    for i, df in enumerate(dfList):
        cols[i] += [pad] * (maxArrayLen - len(cols[i])) #padding the lists to make them have equal lengths 
        nulls[i] += [None] *  (maxArrayLen - len(nulls[i]))
        dictCsvs[datasetNames[i]] = cols[i]
        dictCsvs[dtypesCol + str(i+1)] = nulls[i]
    
    return pd.DataFrame(dictCsvs)

In [112]:
datasetsDtypes(dfAll, lstCsvNames)

,acquisitions,dtypes_1,companies,dtypes_2,investments,dtypes_3,rounds,dtypes_4
0,company_permalink,object,company_permalink,object,company_permalink,object,company_permalink,object
1,company_name,object,company_name,object,company_name,object,company_name,object
2,company_category_list,object,homepage_url,object,company_category_list,object,company_category_list,object
3,company_country_code,object,company_category_list,object,company_country_code,object,company_country_code,object
4,company_state_code,object,funding_total_usd,object,company_state_code,object,company_state_code,object
5,company_region,object,status,object,company_region,object,company_region,object
6,company_city,object,company_country_code,object,company_city,object,company_city,object
7,acquirer_permalink,object,company_state_code,object,investor_permalink,object,funding_round_permalink,object
8,acquirer_name,object,company_region,object,investor_name,object,funding_round_type,object
9,acquirer_category_list,object,company_city,object,investor_country_code,object,funding_round_code,object


## `rounds` Dataset

In [113]:
dfAllCols.iloc[:, -2:]

,investments,rounds
0,company_permalink,company_permalink
1,company_name,company_name
2,company_category_list,company_category_list
3,company_country_code,company_country_code
4,company_state_code,company_state_code
5,company_region,company_region
6,company_city,company_city
7,investor_permalink,funding_round_permalink
8,investor_name,funding_round_type
9,investor_country_code,funding_round_code


In [114]:
invests.shape, rounds.shape

((168647, 18), (114949, 12))

In [115]:
invests.head()

,company_permalink,company_name,company_category_list,company_country_code,company_state_code,company_region,company_city,investor_permalink,investor_name,investor_country_code,investor_state_code,investor_region,investor_city,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd
0,/organization/0-6-com,0-6.com,Curated Web,CHN,22,Beijing,Beijing,/organization/dt-capital-partners,DT Capital Partners,CHN,23,Shanghai,Shanghai,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,3/19/2008,2000000.0
1,/organization/004-technologies,004 Technologies,Software,USA,IL,"Springfield, Illinois",Champaign,/organization/venturecapital-de,VCDE Venture Partners,DEU,5,Frankfurt,Frankfurt,/funding-round/1278dd4e6a37fa4b7d7e06c21b3c1830,venture,NaN,7/24/2014,NaN
2,/organization/01games-technology,01Games Technology,Games,HKG,NaN,Hong Kong,Hong Kong,/organization/cyberport-hong-kong,Cyberport Hong Kong,HKG,NaN,Hong Kong,Hong Kong,/funding-round/7d53696f2b4f607a2f2a8cbb83d01839,undisclosed,NaN,7/1/2014,41250.0
3,/organization/0xdata,H2O.ai,Analytics,USA,CA,SF Bay Area,Mountain View,/organization/capital-one,Capital One,USA,VA,"Washington, D.C.",Mclean,/funding-round/3bb2ee4a2d89251a10aaa735b1180e44,venture,B,11/9/2015,20000000.0
4,/organization/0xdata,H2O.ai,Analytics,USA,CA,SF Bay Area,Mountain View,/organization/nexus-venture-partners,Nexus Venture Partners,USA,CA,SF Bay Area,Menlo Park,/funding-round/383a9bd2c04f7038bb543ccef5ba3eae,seed,NaN,5/22/2013,3000000.0


In [116]:
rounds.head()

,company_permalink,company_name,company_category_list,company_country_code,company_state_code,company_region,company_city,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd
0,/organization/-fame,#fame,Media,IND,16,Mumbai,Mumbai,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,1/5/2015,10000000.0
1,/organization/-qounter,:Qounter,Application Platforms|Real Time|Social Network...,USA,DE,DE - Other,Delaware City,/funding-round/22dacff496eb7acb2b901dec1dfe5633,venture,A,10/14/2014,NaN
2,/organization/-qounter,:Qounter,Application Platforms|Real Time|Social Network...,USA,DE,DE - Other,Delaware City,/funding-round/b44fbb94153f6cdef13083530bb48030,seed,NaN,3/1/2014,700000.0
3,/organization/-the-one-of-them-inc-,"(THE) ONE of THEM,Inc.",Apps|Games|Mobile,NaN,NaN,NaN,NaN,/funding-round/650b8f704416801069bb178a1418776b,venture,B,1/30/2014,3406878.0
4,/organization/0-6-com,0-6.com,Curated Web,CHN,22,Beijing,Beijing,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,3/19/2008,2000000.0


In [117]:
len(invests["company_name"].unique()), len(rounds["company_name"].unique())

(44579, 66103)

Since there are companies in `rounds` dataset that are not in `investments` dataset, <br>
therefore we should keep `rounds` dataset for further analysis.

## `companies` Dataset

In [118]:
companies.sample(5).T

,50945,63168,52191,16601,59967
company_permalink,/organization/shanghai-huan-xiong-information-...,/organization/wavesat,/organization/skydeck,/organization/drug-response-dx,/organization/twentyfour6
company_name,Huan Xiong,Wavesat,Skydeck,Drug Response Dx,TwentyFour6
homepage_url,http://www.ca8.com.cn,http://www.wavesat.com,http://skydeck.com,http://www.drdx.de,http://www.twentyfour6.com
company_category_list,Games,Semiconductors,Android|Messaging|Mobile|Networking|Photograph...,Biotechnology,Advertising
funding_total_usd,1800000,22110000,4000000,-,465000
status,operating,acquired,operating,operating,closed
company_country_code,CHN,CAN,USA,DEU,USA
company_state_code,23,QC,CA,11,IL
company_region,Shanghai,Montreal,SF Bay Area,DEU - Other,Chicago
company_city,Shanghai,Montréal,Palo Alto,Hennigsdorf,Chicago


In [119]:
companies.isnull().sum()

company_permalink            0
company_name                 1
homepage_url              5058
company_category_list     3148
funding_total_usd            0
status                       0
company_country_code      6958
company_state_code        8547
company_region            8030
company_city              8028
funding_rounds               0
founded_at               15221
first_funding_at            24
last_funding_at              0
dtype: int64

we will need to remove the none values of the following columns <br>
based on the question that we are trying to gain insight for: <br>
1.	Can the factors that affect a start-up’s growth be determined?
    -	For this: We are initially interested in <br> `funding_total_usd`, `status`, `funding_rounds`, `first_funding_at`, and `last_funding_at`
    -   So we could remove the 24 null values of `first_funding_at` since they are so small <br>
        compared to the rest of the dataset
2.  Do taxes play an important factor in deciding a start-up’s success?



test